# The Art of Prolog
1986年MIT Pressから出版されたPrologの教科書 The Art of Prolog は、私の前職である株式会社 構造計画研究所から1988年1月1日に「Prologの技芸」というタイトルで日本語訳が出版されました。

残念ながらPrologの技芸は絶版となり、Amazonの古本でも12,800円と高額で販売されています。

幸運なことに、英語の原本の方は、MIT Book pressのサーバからダウンロードすることができます。
- https://mitp-content-server.mit.edu/books/content/sectbyfn/books_pres_0/1407/1407.pdf?dl=1

2024年になって、推論エンジンの原理と応用を再度確認するために、「Prologの技芸」の例題をpythonにポーティングした推論エンジンと簡易パーサを使ってトレースしました。

さらに、swi prologを使って、23章の「コンパイラ」をトレースしました。以下のノートはその時の記録に加筆し、整理したものです。

## ノートブック実行の注意
最初に、コマンドパレットを表示し、"Select Notebook Kernel"を実行し、"Prolog"を選択します。
これで、code cellの内容がswi prologによって処理されます。

## 差分リストの表現
23章のコンパイラーでは、16章に出てくる差分リスト表現（X\Y）が使われていいますが、これを[X, Y]としても実行可能かどうかを試します。

差分リストのメリットは、リストのコピーが不要で、tail(尾部)は、ポインターとして働くことです。

## １６章　限定節文法による構文解析

### 文脈自由文法
16.1の文脈自由文法を定義します。

気を付ける点は、終端記号はカギ括弧で括る。
トップダウン文法でさりげなく、右再帰構文になっていることに注意！

In [1]:
% Figure 16.1 簡単な文脈自由文法
sentence --> noun_phrase, verb_phrase.
noun_phrase --> determiner, noun_phrase2.
noun_phrase --> noun_phrase2.
noun_phrase2 --> adjective, noun_phrase2.
noun_phrase2 --> noun.
verb_phrase --> verb.
verb_phrase --> verb, noun_phrase.
% 語彙
determiner --> [the].   adjective --> [decorated].
determiner --> [a].
noun --> [pieplate].    verb --> [surprise].
noun --> [surprise].

% Asserting clauses for user:sentence/2


% Asserting clauses for user:noun_phrase/2


% Asserting clauses for user:noun_phrase2/2


% Asserting clauses for user:verb_phrase/2


% Asserting clauses for user:determiner/2


% Asserting clauses for user:adjective/2


% Asserting clauses for user:noun/2


% Asserting clauses for user:verb/2


分脈自由文法は、S\S0（ヘッドとテール）の差分リストを引数とする。prolog文に変換されます。
```prolog
sentence --> noun_phrase, verb_phrase.
```
これをそのままprolog規則に変換すると
```prolog
sentence(S) :- append(NP, VP, S), noun_pharase(NP), verb_phrase(VP).
```
となるが、appendの処理効率がわるいため、差分リストで表すと以下の様になります。
```prolog
sentence(S\S0) :- noun_phrase(S\S1), verb_phrase(S1\S0).
```
さらにAs\Bsは、headとtailのリストに分解されるので、
```prolog
sentence(S, S0) :- noun_phrase(S, S1), verb_phrase(S1, S0).
```
となります。

listing関数を使って、sentenceの変換結果を見ると、上記の変換が行われていることが確認できます。

In [2]:
?- listing(sentence).

:- dynamic sentence/2.

sentence(A, C) :-
    noun_phrase(A, B),
    verb_phrase(B, C).


true

### 文法規則からProlog節への変換
swi-prologは、すでに文法規則からProlog節への変換が組み込まれていますが、Program 16.2の例題の動作をswi-prologで動かしてみます。

sequenceはswi-prologのライブラリをそのまま使用します。

In [3]:
% translate(GrammerRule, PrologClause) :-
%   PrologClauseは、文脈自由文法の規則GrammerRuleと等価なPrologプログラムである。
translate((Lhs --> Rhs), (Head :- Body)) :- 
    translate(Lhs, Head, [Xs, Ys]), translate(Rhs, Body, [Xs, Ys]).
translate((A, B), (A1, B1), [Xs, Ys]) :- 
    translate(A, A1, [Xs, Xs1]), translate(B, B1, [Xs1, Ys]).
translate(A, A1, S) :-
    non_terminal(A), functor(A1, A, 1), arg(1, A1, S).
translate(Xs, true, S) :-
    terminals(Xs), sequence(Xs, S).
non_terminal(A) :- atom(A).
terminals([X|Xs]).
%sequence([X|Xs], [[X|S], S0]) :- sequence(Xs, [S, S0]).
%sequence([], [Xs, Xs]).

% Asserting clauses for user:translate/2


% Asserting clauses for user:translate/3


% Asserting clauses for user:terminals/1


動作確認として、以下の自由文脈文法をProlog節に変換してみます。
```
s --> n, v
```

In [4]:
?- translate((s --> n, v), (H :- B)), print(H), print(:-), print(B).

s([_6856,_6862]):-n([_6856,_6896]),v([_6896,_6862])

H = s([_6856,_6862]),
B = n([_6856,_6896]),v([_6896,_6862])

出力されたProlog節を実行し、listing関数で表示してみると、期待通り以下の出力が返されます。
```
s([A, C]) :- n([A, B]), v([B, C]).

In [5]:
s([_9348,_9354]):-n([_9348,_9388]),v([_9388,_9354])

% Asserting clauses for user:s/1


In [6]:
?- listing(s).

:- dynamic s/1.

s([A, C]) :-
    n([A, B]),
    v([B, C]).


true

### 差分リストの表現
15章のProgram 15.2では差分リストを使って、以下のようにappend_dlが定義されています。

```prolog
append_dl(As, Bs, Cs) :- append_dl(Xs\Ys, Ys\Zs, Xs\Zs).
```

このX\Yの部分を[X, Y]に置き換えて、動きを見てみます。

In [7]:
append_dl([Xs, Ys], [Ys, Zs], [Xs, Zs]).
?- append_dl([[a, b, c|Xs], Xs], [[1, 2], []], Ys).

% Asserting clauses for user:append_dl/3


Xs = [1,2],
Ys = [[a,b,c,1,2],[]]

テキストの本文に?- append_dl([a, b, c|Xs]\Xs, [1, 2]\[], Ys)の結果が(Xs=[1,2], Ys=[a, b, c, 1, 2]\[])と記述があります。

これは上記の結果と合致しています。

### 16.5の例題
数字を英語読みに変換する16.5の例題をswi-prologで試してみます。

In [8]:
number(0) --> [zero].
number(N) --> xxx(N).
xxx(N) --> digit(D), [hundred], rest_xxx(N1), { N is D*100 + N1}.
xxx(N) --> xx(N).
rest_xxx(0) --> [].
rest_xxx(N) --> [and], xx(N).

xx(N) --> digit(N).
xx(N) --> teen(N).
xx(N) --> tens(T), rest_xx(N1), {N is T + N1}.

rest_xx(0) --> [].
rest_xx(N) --> digit(N).

digit(1) --> [one].     teen(10) --> [ten].
digit(2) --> [two].     teen(11) --> [eleven].
digit(3) --> [three].   teen(12) --> [twelve].
digit(4) --> [four].    teen(13) --> [thirteen].
digit(5) --> [five].    teen(14) --> [fourteen].
digit(6) --> [six].     teen(15) --> [fifteen].
digit(7) --> [seven].   teen(16) --> [sixteen].
digit(8) --> [eight].   teen(17) --> [seventeen].
digit(9) --> [nine].    teen(18) --> [eighteen].
                        teen(19) --> [nineteen].
tens(20) --> [twenty].
tens(30) --> [thirty].
tens(40) --> [fourty].
tens(50) --> [fifty].
tens(60) --> [sixty].
tens(70) --> [seventy].
tens(80) --> [eighty].
tens(90) --> [ninety].


% Asserting clauses for user:number/3


% Asserting clauses for user:xxx/3


% Asserting clauses for user:rest_xxx/3


% Asserting clauses for user:xx/3


% Asserting clauses for user:rest_xx/3


% Asserting clauses for user:digit/3


% Asserting clauses for user:teen/3


% Asserting clauses for user:tens/3


試しに66をnumberに渡してみます。期待通りsixty sixがリストして返されました。

In [9]:
?- number(66, Ns, []).

Ns = [sixty,six]

## 和と積の演算をDCGで表す
和と積の演算をDefinite Clause Grammer(DCG)で実装してみましょう。
DCGへの入力は、終端子、非終端子の並びをカンマで区切ったリストで与えます。

以下が、和と積の演算（優先順位を考慮）をDCGで定義したものです。
ここでの表現で空（空のリスト）または、演算子とexprの連結として表現している点です。

In [10]:
expr --> term, addterm.
addterm --> [].
addterm --> [+], expr.
term --> factor, multfactor.
multfactor --> [].
multfactor --> [*], term.
factor --> [I], {integer(I)}.
factor --> ['('], expr, [')'].

% Asserting clauses for user:expr/2


% Asserting clauses for user:addterm/2


% Asserting clauses for user:term/2


% Asserting clauses for user:multfactor/2


% Asserting clauses for user:factor/2


試しに4*5+1がexprとして成り立っているかexprに問い合わせてみましょう。

In [11]:
?- expr([4,*,5,+,1], []).

true

### 計算結果を返す
電卓のように計算結果を返すようにDCGを修正してみましょう。
途中の{}で囲まれた箇所がprologの実行文として処理されます。

Xが項(term)でYがaddtermなら、NにX + Yの結果をセットすると言うのが、最初の文の読み方となります。

```prolog
expr(N) --> term(X), addterm(Y), { N is X + Y}.
```

prologの処理を追加したDCGは、以下のようになります。

In [12]:
expr(N) --> term(X), addterm(Y), { N is X + Y}.
addterm(0) --> [].
addterm(N) --> [+], expr(N).
term(N) --> factor(X), multfactor(Y), {N is X * Y}.
multfactor(1) --> [].
multfactor(S) --> [*], term(S).
factor(I) --> [I], {integer(I)}.

% Asserting clauses for user:expr/3


% Asserting clauses for user:addterm/3


% Asserting clauses for user:term/3


% Asserting clauses for user:multfactor/3


% Asserting clauses for user:factor/3


この文法のprologに変換した結果をlisting関数を使って確認してみましょう。

exprの2つ目の定義には、期待通りC is D+Eが計算されています。

In [13]:
?- listing(expr).

:- dynamic expr/2.

expr(A, C) :-
    term(A, B),
    addterm(B, C).

:- dynamic expr/3.

expr(C, A, F) :-
    term(D, A, B),
    addterm(E, B, G),
    C is D+E,
    F=G.


true

それではexprを使って計算結果を見てみましょう。
積が和よりも優先して処理され、演算結果が７になっていることが確認できました。

In [14]:
?- expr(Y, [1,+,2,*,3], []).

Y = 7

### ANTRLの四則演算をprologで実装
私のブログにANTRLを使って四則演算をする例題があります。
- <a href="https://take-pwave.sakura.ne.jp/index.php?antlr%2FANTLRWorks%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B">antlr/ANTLRWorksを使ってみる</a>

ここで紹介している文法をprologにDCGに書き換えてみます。

```
expression
	: product ('+' product | '-' product)*
	;

product
	: power ('*' power | '/' power)*
	;
power
	: factor ('^' power)?
	;
factor
	: IDENTIFIER
	| CONSTANT
	| '(' expression ')'
	;
```

In [15]:
expression(N) -->
    product(X), 
    ( [],
        {N is X}
    | [+], expression(Y), 
        {N is X + Y} 
    | [-], expression(Y), 
        {N is X - Y}
    ).
product(N) -->
    power(X), 
    ( [],
        { N is X}
    | [*], product(Y),
        { N is X * Y}
    | [/], product(Y),
        { N is X / Y}
    ).
power(N) -->
    factor(X),
    ([],
        { N is X}
    | ['^'], power(Y),
        { N is X**Y}
    ).
factor(I) --> [I], {number(I)}.
    

% Asserting clauses for user:expression/3


% Asserting clauses for user:product/3


% Asserting clauses for user:power/3


Previously defined clauses of user:factor/3 were retracted:
:- dynamic factor/3.

factor(A, [A|C], B) :-
    integer(A),
    B=C.

% Asserting clauses for user:factor/3


これで1+2^3*4を計算してみましょう。

優先順位的には、1 + ((2^3)*4)となりますので、1 + (8*4) = 33となります。

In [16]:
?- expression(N, [1, +, 2, '^', 3, *, 4], []).

N = 33